In [ ]:
ort numpy as np
import pandas as pd
import datetime
from six import StringIO

In [ ]:
#initialize
def initialize(account):
    #initialized stock pool
    get_iwencai('沪深300')
    #set the maximum holding
    #account.n = 30
    #stock candidates
    account.stock_pool = []
    #sub_portfolio
    #set_subportfolios([{'cash':1000000,'type':'stock'},{'cash':1000000,'type':'future'}])
    account.max_stocks = 30
    #set adjustment period_adjust at the second trading day of every month
    schedule_function(func=reallocation,date_rule=date_rules.month_start(1))

    
# get the data before market opening
def before_trading_start(account,data):
    # set the holding number
    #n = account.n
    # get the previous trading date
    date = get_last_datetime()
    # get the historical stock data
    pn_data = get_price(account.iwencai_securities,start_date = None,end_date = date,
                        fre_step = '1d',fields = ['close','prev_close'],skip_paused = True,
                        fq = None,bar_count = 30,is_panel = 1)
    pn_data1 = get_price(['000001.SH'],start_date = None,end_date = date,fre_step = '1d',
                         fields = ['turnover_rate'],skip_paused = True,fq = None,bar_count = 30,is_panel = 1)

    # example_alpha
    alpha = Alphas(pn_data, pn_data1)
    #call function factor01 to return dic,put into df
    df = alpha.factor01()
    #convert dic to df, sort by key
    df =pd.DataFrame(df).sort_values(by='cov',ascending=False)
    log.info(df)
    #choose first 15 stocks
    stock1 = df.security[:15];
    log.info(stock1)
    #choose bottom 15 stocks
    stock2 = df.security[-15:];
    log.info(stock2)

    #put the selected stocks into stock pool
    account.stock_pool = pd.concat([stock1, stock2])
    #account.stock_pool = stock2
    
    
def reallocation(account, data):
    #sell all stocks before adjustment day
    for security in list(account.positions):
        order_target(security, 0)
    #reallocate
    cash = account.cash/30
    #buy new
    for security in account.stock_pool:
        order_target_value(security, cash)

    
#define a Alpha class
class Alphas(object):
    def __init__(self, pn_data, pn_data1):
        #get close price
        self.close = pn_data['close']
        #get previous close
        self.prev_close = pn_data['prev_close']
        #get 000001.SH’s turnover data
        self.tv = pn_data1['turnover_rate']
        #define a dic,the rank of the stock, cov(turnover,return)
        self.df = {'security':[], 'cov':[]}
    def factor01(self):
        #process the factor
        m1 = np.mean(self.tv)
        s1 = np.std(self.tv)
        # standardized
        self.tv = (self.tv-m1)/s1
        #calculate coviarance
        #calculate return
        returnn = np.log(self.close)-np.log(self.prev_close)
        #loop
        for security in returnn:
            cov = np.cov(returnn[security],self.tv['000001.SH'])[0,1]

        #remove nan
        if np.isnan(cov)==False:
            #append ticker name
            self.df['security'].append(security)
            #add cov
            self.df['cov'].append(cov)
            #retrun df
        return self.df
